# New Narratives for old Buildings

Finding narratives through bigram and trigram analysis

In [1]:
# import necessary libraries
import os, re, csv
import pandas as pd
import numpy as np
import gensim, nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import islice
from nltk.corpus import stopwords
import spacy
import string
import pickle

# 1. Loading the datasets (all languages)

Use google drive storage or local copy of data

In [2]:
#Google drive:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
path = 'datasets/'

In [4]:
# Country code: change here between 'NL' and 'UK'
cc_list = ['NL', 'UK', 'DE', 'FR']

# NL

## 0.1 Loading the dataset: NL

Use google drive storage or local copy of data

In [ ]:
raw_data_file = path+cc_list[0]+'_dataset_website-content-crawler.json'

In [ ]:
# Import json data from Aipfy scraping
df=pd.read_json(raw_data_file)

# Print the DataFrame
df

,url,crawl,metadata,screenshotUrl,text,markdown,debug
0,http://weldam.nl/,{'loadedUrl': 'http://weldam.nl/english/home.h...,{'canonicalUrl': 'http://weldam.nl/english/hom...,NaN,"Introduction - Weldam\nIntroduction\nWeldam, s...",# Introduction - Weldam\n\n### Introduction\n\...,{'requestHandlerMode': 'browser'}
1,http://weldam.nl/nederlands.html,{'loadedUrl': 'http://weldam.nl/nederlands.htm...,{'canonicalUrl': 'http://weldam.nl/nederlands....,NaN,Nederlands - Weldam\nCopyright Landgoed Weldam...,# Nederlands - Weldam\n\nCopyright Landgoed We...,{'requestHandlerMode': 'browser'}
2,http://weldam.nl/nederlands/beginpagina/test-2...,{'loadedUrl': 'http://weldam.nl/nederlands/beg...,{'canonicalUrl': 'http://weldam.nl/nederlands/...,NaN,Test 1.2 - Weldam\nCopyright Landgoed Weldam 2...,# Test 1.2 - Weldam\n\nCopyright Landgoed Weld...,{'requestHandlerMode': 'browser'}
3,https://www.huisdoorn.nl/,"{'loadedUrl': 'https://www.huisdoorn.nl/', 'lo...","{'canonicalUrl': 'https://www.huisdoorn.nl/', ...",NaN,Ontdek de geschiedenis - Museum Huis Doorn\nDe...,# Ontdek de geschiedenis - Museum Huis Doorn\n...,{'requestHandlerMode': 'browser'}
4,https://www.museumdefundatie.nl/,{'loadedUrl': 'https://www.museumdefundatie.nl...,{'canonicalUrl': 'https://www.museumdefundatie...,NaN,Museum de FundatieTwitter Widget Iframe\nMuseu...,# Museum de FundatieTwitter Widget Iframe\n\n!...,{'requestHandlerMode': 'browser'}
...,...,...,...,...,...,...,...
10946,https://www.kasteelheeswijk.nl/en/my-bookings/,{'loadedUrl': 'https://www.kasteelheeswijk.nl/...,{'canonicalUrl': 'https://www.kasteelheeswijk....,NaN,Mijn boekingenreCAPTCHA\nMijn boekingen\n[book...,# Mijn boekingenreCAPTCHA\n\n## Mijn boekingen...,{'requestHandlerMode': 'browser'}
10947,https://www.kasteelheeswijk.nl/de/2023/blog/tr...,{'loadedUrl': 'https://www.kasteelheeswijk.nl/...,{'canonicalUrl': 'https://www.kasteelheeswijk....,NaN,Kasteel trouwen | Unieke trouwlocatie\nHet jaw...,# Kasteel trouwen | Unieke trouwlocatie\n\nHet...,{'requestHandlerMode': 'browser'}
10948,https://www.kasteelheeswijk.nl/en/2023/blog/tr...,{'loadedUrl': 'https://www.kasteelheeswijk.nl/...,{'canonicalUrl': 'https://www.kasteelheeswijk....,NaN,Kasteel trouwen | Unieke trouwlocatie\nHet jaw...,# Kasteel trouwen | Unieke trouwlocatie\n\nHet...,{'requestHandlerMode': 'browser'}
10949,https://www.kasteelheeswijk.nl/de/plan-je-bezo...,{'loadedUrl': 'https://www.kasteelheeswijk.nl/...,{'canonicalUrl': 'https://www.kasteelheeswijk....,NaN,Lunchen Bij de Barones | Gezellig bij Kasteel ...,# Lunchen Bij de Barones | Gezellig bij Kastee...,{'requestHandlerMode': 'browser'}


In [ ]:
# correcting the data to include a missing castle (why was it missing?)
df_missing = pd.read_json('datasets/NL_LG_dataset_website-content-crawler_2025-02-06_09-40-33-880.json')
result = pd.concat([df, df_missing])

In [ ]:
df = result
df.shape

(11000, 7)

###  0.1.1 Selecting data, sorting and deduplicating

In [ ]:
# select only two columns for analysis: url and text
df=df[['url','text']]
df.head()

,url,text
0,http://weldam.nl/,"Introduction - Weldam\nIntroduction\nWeldam, s..."
1,http://weldam.nl/nederlands.html,Nederlands - Weldam\nCopyright Landgoed Weldam...
2,http://weldam.nl/nederlands/beginpagina/test-2...,Test 1.2 - Weldam\nCopyright Landgoed Weldam 2...
3,https://www.huisdoorn.nl/,Ontdek de geschiedenis - Museum Huis Doorn\nDe...
4,https://www.museumdefundatie.nl/,Museum de FundatieTwitter Widget Iframe\nMuseu...


Join all pages from a domain to an entry in the analysis: add a new column which will contain only the main domain name.

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\w+\.)*\w+\.\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Load the list of domains from a csv file:
cc_column = cc_list[0]+' domains'
#print(cc_column)

urls = pd.read_csv('url_lists/'+cc_list[0]+'_urls.csv')[cc_column].values.tolist()

# Extract main domains from nl_urls
domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}

# Check if main domains in list_of_links match any domain in nl_domains
matching_links = [link for link in df.url if extract_main_domain(link) in domains]

In [ ]:
# this cell can be skipped, it is only for verification

# check how many lines in the dataframe have a matching link to the list of urls
print(len(matching_links))

10951


In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column
df['domain'] = df['url'].apply(extract_main_domain)
df.head()

/var/folders/ys/vp5zrwv13sx9bt05xzqj9pzh0000gn/T/ipykernel_22392/597993017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain'] = df['url'].apply(extract_main_domain)


,url,text,domain
0,http://weldam.nl/,"Introduction - Weldam\nIntroduction\nWeldam, s...",eldam.nl
1,http://weldam.nl/nederlands.html,Nederlands - Weldam\nCopyright Landgoed Weldam...,eldam.nl
2,http://weldam.nl/nederlands/beginpagina/test-2...,Test 1.2 - Weldam\nCopyright Landgoed Weldam 2...,eldam.nl
3,https://www.huisdoorn.nl/,Ontdek de geschiedenis - Museum Huis Doorn\nDe...,huisdoorn.nl
4,https://www.museumdefundatie.nl/,Museum de FundatieTwitter Widget Iframe\nMuseu...,museumdefundatie.nl


In [ ]:
df=df.sort_values('url', ascending=False)

In [ ]:
# Deduplicate entries
duplicate_rows = df[df.duplicated(subset=['text'], keep=False)]
for index, row in duplicate_rows.iterrows():
    print(f"URL: {row['url']}\nText: {row['text'][0:50]}\n")

In [ ]:
df = df[~df.index.isin(duplicate_rows.index.tolist())]
df.shape

In [ ]:
# consider removing pages that contain '404' and similar messages; may remove pages with content!!!
not_found = ['Helaas, hier is iets mis gegaan', 'Pagina niet gevonden', 'Email Protection', '404']
rows_to_drop = []
find_not_found = lambda df, not_found: [index for index, row in df.iterrows() if any(item in row['text'] for item in not_found)]
rows_to_drop = find_not_found(df, not_found)

print(len(rows_to_drop), rows_to_drop)

311 [4231, 4228, 9895, 9889, 8100, 768, 772, 767, 771, 757, 759, 769, 760, 770, 8166, 8168, 8167, 8169, 1923, 5289, 5290, 10721, 5985, 9772, 5930, 5992, 9535, 9546, 5993, 5995, 4014, 5987, 9634, 5989, 3737, 9539, 9805, 9637, 6479, 2291, 6356, 10724, 10764, 10767, 10766, 10768, 10765, 10769, 9787, 9788, 9821, 9822, 9897, 5551, 5586, 6059, 5553, 6058, 6060, 5550, 5579, 7746, 10927, 10938, 9526, 3594, 9631, 9630, 9522, 3577, 9652, 9525, 9651, 9653, 9524, 3582, 2999, 9050, 9049, 3585, 9065, 8956, 3765, 3277, 9001, 2804, 9056, 3259, 3252, 3014, 3258, 3257, 3241, 4764, 4718, 980, 4663, 1017, 4691, 10683, 4703, 5699, 5614, 5659, 5667, 5662, 9521, 3540, 6056, 6057, 6772, 6816, 6778, 6792, 6931, 6814, 6913, 6738, 6739, 6740, 2476, 6777, 7239, 6987, 6986, 6985, 2547, 6838, 6837, 6836, 2510, 6020, 9127, 8291, 8289, 987, 986, 996, 984, 983, 988, 990, 989, 991, 1027, 1007, 998, 1021, 1000, 1001, 1002, 999, 1012, 1005, 1008, 1011, 1013, 1016, 1014, 1020, 1022, 1024, 1032, 1039, 1025, 1028, 1026, 107

In [ ]:
df = df[~df.index.isin(rows_to_drop)]
df.shape

(10689, 3)

### 0.1.2 Close reading one document
(we consider one website as a 'document')

In [ ]:
df.domain.unique()

array(['twickel.nl', 'sypesteyn.nl', 'slotzuylen.nl', 'slotloevestein.nl',
       'schierstins.nl', 'ruinevanbrederode.nl', 'ruine', 'poptaslot.nl',
       'museumnienoord.nl', 'museummartena.nl', 'museumijsselstein.nl',
       'museumdefundatie.nl', 'muiderslot.nl', 'middachten.nl',
       'mensinge.nl', 'menkemaborg.nl', 'landgoedvilsteren.nl',
       'landgoedoldenhof.nl', 'landgoedbouvigne.nl',
       'landgoedboschenvaart.nl', 'kasteelwijlre.nl', 'kasteelwarmelo.nl',
       'kasteelvalkenburg.nl', 'kasteeltuinen.nl',
       'kasteeltuinassumburg.nl', 'kasteelteylingen.nl',
       'kasteelradboud.nl', 'kasteelparkborn.nl', 'kasteelnemerlaer.nl',
       'kasteelkeukenhof.nl', 'kasteelhoensbroek.nl', 'kasteelheeze.nl',
       'kasteelheeswijk.nl', 'kasteelgroeneveld.nl', 'kasteelgeldrop.nl',
       'kasteeldussen.nl', 'kasteelduivenvoorde.nl',
       'kasteeldoornenburg.nl', 'kasteeldever.nl', 'kasteeldehaar.nl',
       'kasteelboxmeer.nl', 'kasteelamerongen.nl', 'huystendonck.nl',
 

In [ ]:
# then combine these into a list of pages
document = df[df['domain'] == 'museumdefundatie.nl']['text'].tolist()

# remove new line characters, count words&characters in each line of the text
clean_document = []
word_count = 0
character_count = 0
for i in document:
    j = i.replace('\n', ' ').replace('\r', '')
    w = len(j.split())
    word_count += w
    character_count +=len(j)
    clean_document.append(j)

In [ ]:
print('word_count:', word_count, '\ncharacter_count:', character_count)
clean_document

word_count: 145629 
character_count: 968608 
the document: ['Museum de FundatieTwitter Widget Iframe Museum de Fundatie owns and manages a vast collection of visual arts. Collection and exhibitions are on display at two locations: Kasteel Het Nijenhuis near Heino and Museum de Fundatie in Zwolle. TICKETS MUSEUM DE FUNDATIE ZWOLLE TICKETS KASTEEL HET NIJENHUIS Museum de Fundatie: Today opened from 11:00 - 17:00 Nijenhuis Castle and the Sculpture Garden:  Today opened from 11:00 - 17:00', 'Sculpture garden - Museum de FundatieTwitter Widget Iframe From: 01 january 2016 Through: 31 december 2024 Location: Kasteel Het Nijenhuis Sculpture garden - near Kasteel het Nijenhuis Please note that from November through mid-March, some of the sculptures in the Sculpture Garden are covered to protect them from the cold. PLEASE NOTE: Kasteel het Nijenhuis will only be accessible via the stationsweg from Heino station/centre Heino in the coming weeks. The castle is not accessible from the Wechterholt.

## 0.2 Counting terms NL

In [ ]:
terms_list_all = ['koloni', 'slaaf', 'slave', 'tot slaaf gemaakt', 'bediende', 'zwarte bediende',
                  'arbeider', 'loon', 'pacht', 'boer', 'boerd', 'indië', 'nederlands-indië', 'de oost', 'het oosten',
                  'voc', 'verenigde oostindische compagnie', 'upstairs-downstairs',
                  'vrouw', 'gender', 'diversiteit', 'diverse','rijk', 'arm', 'inclusie']
#term_list = ['koloni']
#term_list = ['ongelijkheid', 'democratie', 'armoede']
#term_list = ['slavernij', 'slaaf', 'slaven']
term_list = ['diversiteit', 'biodiversiteit', 'inclusiviteit']
#term_list = ['bediende','arbeider']

In [ ]:
df

,url,text,domain,bediende,arbeider
10686,https://www.twickel.nl/hilligenpad,Het Hilligenpad - Stichting Twickel\nHilligenp...,twickel.nl,0,0
16,https://www.twickel.nl/,Home - Stichting Twickel\nAgenda\nLichtjeswand...,twickel.nl,0,0
20,https://www.sypesteyn.nl/,Kasteel Sypesteyn\nVanaf maandag 4 november zi...,sypesteyn.nl,0,0
10,https://www.slotzuylen.nl/,Kasteel - Slot Zuylenslotzuylen - YouTubereCAP...,slotzuylen.nl,0,0
4128,https://www.slotloevestein.nl/zakelijke-events/,Zakelijke events - Slot Loevestein\nZoek je ee...,slotloevestein.nl,0,0
...,...,...,...,...,...
1643,http://slotzuylen.nl/aan-het-hof-van-tuyll-in-...,"Aan het Hof; van Tuyll in dienst van koningen,...",slotzuylen.nl,0,0
1241,http://museumijsselstein.nl/over-het-museum/pr...,Privacy verklaring | Museum IJsselstein\nPriva...,museumijsselstein.nl,0,0
6690,http://landgoedvilsteren.nl/foutje.html/nl/aeo...,"Foutje - Landgoed Vilsteren\nSorry, niet gevon...",landgoedvilsteren.nl,0,0
7786,http://kasteeldehaar.nl/restauratie-lu-xing,Restauratie Lu Xing | Kasteel de Haar\nMomente...,kasteeldehaar.nl,0,0


In [ ]:
for term in term_list:
    df[term] = df['text'].apply(lambda x: x.lower().count(term) if isinstance(x, str) else 0)

# Filter for rows where 'kasteel' appears at least once
mask = (df[term_list] > 0).any(axis=1)

# Filter the DataFrame based on the boolean mask.
df_filtered = df[mask]

# Create the five-column table
result_df = df_filtered[['domain','url','diversiteit', 'biodiversiteit', 'inclusiviteit']]
result_df.to_csv('outputs/2024_filtered_output.csv', index=False)
print(result_df.to_string())

                        domain                                                                                                                                                       url  diversiteit  biodiversiteit  inclusiviteit
8073         slotloevestein.nl                                                                                                       https://www.slotloevestein.nl/werken-op-loevestein/            1               0              0
868          slotloevestein.nl                                                                                                                 https://www.slotloevestein.nl/tegenpolen/            1               0              0
4217         slotloevestein.nl                                                                        https://www.slotloevestein.nl/onderwijsmodules/tentoonstellingsles-verbind-je-mee/            1               0              0
8068         slotloevestein.nl                                                      

In [ ]:
# drop columns to reset
df = df.drop(term_list, axis=1)
df.head()

,url,text,domain
10686,https://www.twickel.nl/hilligenpad,Het Hilligenpad - Stichting Twickel\nHilligenp...,twickel.nl
16,https://www.twickel.nl/,Home - Stichting Twickel\nAgenda\nLichtjeswand...,twickel.nl
20,https://www.sypesteyn.nl/,Kasteel Sypesteyn\nVanaf maandag 4 november zi...,sypesteyn.nl
10,https://www.slotzuylen.nl/,Kasteel - Slot Zuylenslotzuylen - YouTubereCAP...,slotzuylen.nl
4128,https://www.slotloevestein.nl/zakelijke-events/,Zakelijke events - Slot Loevestein\nZoek je ee...,slotloevestein.nl


# UK

## 1.1 Loading the dataset: UK

Use google drive storage or local copy of data

In [48]:
raw_data_file = path+cc_list[1]+'_dataset_website-content-crawler.json'

In [ ]:
# Import json data from Aipfy scraping
df=pd.read_json(raw_data_file)

# Print the DataFrame
df

### 1.1.1 Selecting data, sorting and deduplicating

In [51]:
# select only two columns for analysis: url and text
df=df[['url','text']]
df.head()

,url,text
0,https://www.whittingtoncastle.co.uk/visit-us,Visit Us — Whittington Castle\nOn the Welsh bo...
1,https://www.whittingtoncastle.co.uk/kitchenthe...,Kitchen@theCastle — Whittington Castle\nAt Whi...
2,https://www.whittingtoncastle.co.uk/home,Whittington Castle\nA stunning 12th century ca...
3,https://www.whittingtoncastle.co.uk/weddings-w...,Get in Touch — Whittington CastlereCAPTCHA\nWe...
4,https://www.whittingtoncastle.co.uk/events,Events — Whittington Castle\nUpcoming events\n...


Join all pages from a domain to an entry in the analysis: add a new column which will contain only the main domain name.

In [111]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?m)http(?:s?):\/\/.*?([^\.\/]+?\.[^\.]+?)(?:\/|$)', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [53]:
# Load the list of domains from a csv file:
cc_column = cc_list[1]+' domains'
#print(cc_column)

urls = pd.read_csv(path+'url_lists/'+cc_list[1]+'_urls.csv')[cc_column].values.tolist()

# Extract main domains from nl_urls
domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}

# Check if main domains in list_of_links match any domain in nl_domains
matching_links = [link for link in df.url if extract_main_domain(link) in domains]

In [54]:
# this cell can be skipped, it is only for verification

# check how many lines in the dataframe have a matching link to the list of urls
print(len(matching_links))

6794


In [56]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column
df['domain'] = df['url'].apply(extract_main_domain)
df.head()

,url,text,domain
0,https://www.whittingtoncastle.co.uk/visit-us,Visit Us — Whittington Castle\nOn the Welsh bo...,hittingtoncastle.co.uk
1,https://www.whittingtoncastle.co.uk/kitchenthe...,Kitchen@theCastle — Whittington Castle\nAt Whi...,hittingtoncastle.co.uk
2,https://www.whittingtoncastle.co.uk/home,Whittington Castle\nA stunning 12th century ca...,hittingtoncastle.co.uk
3,https://www.whittingtoncastle.co.uk/weddings-w...,Get in Touch — Whittington CastlereCAPTCHA\nWe...,hittingtoncastle.co.uk
4,https://www.whittingtoncastle.co.uk/events,Events — Whittington Castle\nUpcoming events\n...,hittingtoncastle.co.uk


In [59]:
df=df.sort_values('url', ascending=False)

In [ ]:
# Deduplicate entries
duplicate_rows = df[df.duplicated(subset=['text'], keep=False)]
for index, row in duplicate_rows.iterrows():
    print(f"URL: {row['url']}\nText: {row['text'][0:100]}\n")

In [ ]:
duplicate_rows.index.tolist()

In [69]:
df = df[~df.index.isin(duplicate_rows.index.tolist())]
df.shape

(5974, 3)

### 1.1.2 Check content

In [70]:
df.domain.unique()


array(['oburnabbey.co.uk', 'hittingtoncastle.co.uk', 'entworthcastle.org',
       'arwick', 'tutburycastle.com', 'tonbridgecastle.org',
       'tamworthcastle.co.uk', 'syonpark.co.uk', 'sussexpast.co.uk',
       'strawberryhillhouse.org.uk', 'spencerhouse.co.uk',
       'skiptoncastle.co.uk', 'ryemuseum.co.uk', 'rockinghamcastle.com',
       'raby.co.uk', 'powderham.co.uk', 'pontefractcastle.co.uk',
       'penshurstplace.com', 'pencarrow.co.uk',
       'oxfordcastleandprison.co.uk', 'oakhamcastle.org',
       'nottinghamcastle.org.uk', 'newcastlecastle.co.uk', 'newark',
       'nationaltrust.org.uk', 'muncaster.co.uk', 'lydiardpark.org.uk',
       'ludlowcastle.com', 'lincolncastle.com', 'leeds',
       'lancastercastle.com', 'lancashire.gov.uk',
       'kinnersleycastle.co.uk', 'hrp.org.uk', 'hornbycastle.com',
       'hevercastle.co.uk', 'herstmonceux', 'hemyockcastle.uk',
       'hedinghamcastle.co.uk', 'hants.gov.uk', 'haddonhall.co.uk',
       'grimsthorpe.co.uk', 'godintonhouse.

In [77]:
# extract all rows (lines) where the value of 'domain' is ...
# then combine these into a list of pages
document = df[df['domain'] == 'nationaltrust.org.uk']['text'].tolist()

# remove new line characters, count words&characters in each line of the text
clean_document = []
word_count = 0
character_count = 0
for i in document:
    j = i.replace('\n', ' ').replace('\r', '')
    w = len(j.split())
    word_count += w
    character_count +=len(j)
    clean_document.append(j)

In [ ]:
print('word_count:', word_count, '\ncharacter_count:', character_count)
clean_document

## 1.2 Counting terms UK

In [32]:
terms_list_all = ['coloni','inequality','democracy','poverty','slave','slavery','diversity','inclusion','servant','worker','labourer']
term_list = ['coloni']
#term_list = ['inequality','democracy','poverty']
#term_list = ['slave','slavery']
#term_list = ['diversity', 'inclusion']
#term_list = ['servant','worker','labourer']

In [33]:
df

,url,text,domain,colonial
138,https://www.woburnabbey.co.uk/,Woburn attractions | Woburn\nCustomize Consent...,oburnabbey.co.uk,0
3,https://www.whittingtoncastle.co.uk/weddings-w...,Get in Touch — Whittington CastlereCAPTCHA\nWe...,hittingtoncastle.co.uk,0
0,https://www.whittingtoncastle.co.uk/visit-us,Visit Us — Whittington Castle\nOn the Welsh bo...,hittingtoncastle.co.uk,0
168,https://www.whittingtoncastle.co.uk/trips-and-...,Trips and Tours — Whittington Castle\nWe at Wh...,hittingtoncastle.co.uk,0
166,https://www.whittingtoncastle.co.uk/support-us,Support Us — Whittington Castle\nThe Whittingt...,hittingtoncastle.co.uk,0
...,...,...,...,...
2943,http://www.lancashire.gov.uk/leisure-and-culture/,Leisure and culture - Lancashire County Counci...,lancashire.gov.uk,0
2954,http://www.lancashire.gov.uk/consumer-advice/,Complain about goods or services \nHow to make...,lancashire.gov.uk,0
2947,http://www.lancashire.gov.uk/children-educatio...,"Children, education and families - Lancashire ...",lancashire.gov.uk,0
6641,http://www.birminghammuseums.org.uk/events/cel...,Birmingham Museums | Birmingham Museums\nLoadi...,birminghammuseums.org.uk,0


In [35]:
for term in term_list:
    df[term] = df['text'].apply(lambda x: x.lower().count(term) if isinstance(x, str) else 0)

# Filter for rows where 'kasteel' appears at least once
mask = (df[term_list] > 0).any(axis=1)

# Filter the DataFrame based on the boolean mask.
df_filtered = df[mask]

# Create the five-column table
result_df = df_filtered[['domain','url','coloni']]
result_df.to_csv(path+'outputs/2024_filtered_output.csv', index=False)
print(result_df.to_string())

                           domain                                                                                                                                                                                                url  coloni
5039          skiptoncastle.co.uk                                                                                                                                                   https://www.skiptoncastle.co.uk/skiptonwoods.asp       1
3641  oxfordcastleandprison.co.uk                                                                                                https://www.oxfordcastleandprison.co.uk/about/news/the-horrific-history-of-witch-trials-in-england/       1
2786         nationaltrust.org.uk                                                                                                                                                   https://www.nationaltrust.org.uk/who-we-are/news       2
2797         nationaltrust.org.uk                   

In [34]:
# drop columns to reset
df = df.drop(term_list, axis=1)
df.head()

,url,text,domain
138,https://www.woburnabbey.co.uk/,Woburn attractions | Woburn\nCustomize Consent...,oburnabbey.co.uk
3,https://www.whittingtoncastle.co.uk/weddings-w...,Get in Touch — Whittington CastlereCAPTCHA\nWe...,hittingtoncastle.co.uk
0,https://www.whittingtoncastle.co.uk/visit-us,Visit Us — Whittington Castle\nOn the Welsh bo...,hittingtoncastle.co.uk
168,https://www.whittingtoncastle.co.uk/trips-and-...,Trips and Tours — Whittington Castle\nWe at Wh...,hittingtoncastle.co.uk
166,https://www.whittingtoncastle.co.uk/support-us,Support Us — Whittington Castle\nThe Whittingt...,hittingtoncastle.co.uk


# DE

## 2.1 Loading the dataset: DE

Use google drive storage or local copy of data

In [104]:
raw_data_file = path+cc_list[2]+'_dataset_website-content-crawler.json'

In [ ]:
# Import json data from Aipfy scraping
df=pd.read_json(raw_data_file)

# Print the DataFrame
df

### 2.1.1 Selecting data, sorting and deduplicating

In [106]:
# select only two columns for analysis: url and text
df=df[['url','text']]
df.head()

,url,text
0,http://www.schlossparkmarisfeld.de/,Schlosspark Marisfeld - Schlosspark Marisfeld\...
1,http://www.schlossparkmarisfeld.de/information...,Informationen zur Geschichte des Schlossparks\...
2,http://www.schlossparkmarisfeld.de/information...,Der Schlosspark Marisfeld - Kleine Parkgeschic...
3,http://www.schlossparkmarisfeld.de/information...,Der Förderverein stellt sich vor\nDer aktuelle...
4,http://www.schlossparkmarisfeld.de/information...,Die Vereinssatzung - Schlosspark Marisfeld\nFa...


Join all pages from a domain to an entry in the analysis: add a new column which will contain only the main domain name.

In [107]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?m)http(?:s?):\/\/.*?([^\.\/]+?\.[^\.]+?)(?:\/|$)', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [108]:
# Load the list of domains from a csv file:
cc_column = cc_list[2]+' domains'
#print(cc_column)

urls = pd.read_csv(path+'url_lists/'+cc_list[2]+'_urls.csv')[cc_column].values.tolist()

# Extract main domains from nl_urls
domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}

# Check if main domains in list_of_links match any domain in nl_domains
matching_links = [link for link in df.url if extract_main_domain(link) in domains]

In [110]:
# this cell can be skipped, it is only for verification

# check how many lines in the dataframe have a matching link to the list of urls
print(len(matching_links))

10198


In [112]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column
df['domain'] = df['url'].apply(extract_main_domain)
df.head()

<ipython-input-112-78111174b86d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain'] = df['url'].apply(extract_main_domain)


,url,text,domain
0,http://www.schlossparkmarisfeld.de/,Schlosspark Marisfeld - Schlosspark Marisfeld\...,schlossparkmarisfeld.de
1,http://www.schlossparkmarisfeld.de/information...,Informationen zur Geschichte des Schlossparks\...,schlossparkmarisfeld.de
2,http://www.schlossparkmarisfeld.de/information...,Der Schlosspark Marisfeld - Kleine Parkgeschic...,schlossparkmarisfeld.de
3,http://www.schlossparkmarisfeld.de/information...,Der Förderverein stellt sich vor\nDer aktuelle...,schlossparkmarisfeld.de
4,http://www.schlossparkmarisfeld.de/information...,Die Vereinssatzung - Schlosspark Marisfeld\nFa...,schlossparkmarisfeld.de


In [ ]:
df=df.sort_values('url', ascending=False)

In [113]:
# Deduplicate entries
duplicate_rows = df[df.duplicated(subset=['text'], keep=False)]
for index, row in duplicate_rows.iterrows():
    print(f"URL: {row['url']}\nText: {row['text'][0:100]}\n")

Streaming output truncated to the last 5000 lines.

URL: https://www.neuschwanstein.de/deutsch/schloss/speise.htm#navigation
Text: Bayerische Schlösserverwaltung | Schloss Neuschwanstein | Rundgang
Inhalt:
Speisezimmer 
Über ein ei

URL: https://www.neuschwanstein.de/deutsch/schloss/index.htm#mm-6
Text: Bayerische Schlösserverwaltung | Neuschwanstein | Schloss Neuschwanstein heute
Navigation:
Neuschwan

URL: https://www.neuschwanstein.de/deutsch/schloss/speise.htm#main-content
Text: Bayerische Schlösserverwaltung | Schloss Neuschwanstein | Rundgang
Inhalt:
Speisezimmer 
Über ein ei

URL: https://www.neuschwanstein.de/deutsch/schloss/hauskap.htm#main-content
Text: Bayerische Schlösserverwaltung | Schloss Neuschwanstein | Rundgang
Inhalt:
Hauskapelle
Die an das Sc

URL: https://www.neuschwanstein.de/deutsch/schloss/hauskap.htm#mainNav
Text: Bayerische Schlösserverwaltung | Schloss Neuschwanstein | Rundgang
Inhalt:
Hauskapelle
Die an das Sc

URL: https://www.neuschwanstein.de/deutsch/schl

In [115]:
len(duplicate_rows.index.tolist())

5332

In [116]:
df = df[~df.index.isin(duplicate_rows.index.tolist())]
df.shape

(5107, 3)

### 2.1.2 Check content

In [117]:
df.domain.unique()


array(['schlossparkmarisfeld.de', 'schloss-alsbach.org',
       'nordkirchen.net', 'burghohenzollern.com',
       'heimatverein-schnaittach.com', 'schlossguestrow.de',
       'schlosshomburg.de', 'schloss-hundisburg.de', 'artburg.de',
       'schloss-erbach.de', 'die-brandenburg.de', 'burg-querfurt.de',
       'kaiserburgnuernberg.de', 'schloss-braunfels.de',
       'kasteelbentheim.nl', 'burggreifenstein.de',
       'schlossheidelberg.de', 'hambacherschloss.de',
       'schloss-gottorf.de', 'bergedorfer-museumslandschaft.de',
       'museum-schloss-fellenberg.de', 'ludwigsburg-mv.de',
       'museum-schloss-schoenebeck.de', 'schlossgoseck.de',
       'landesmuseum.de', 'asserburg-anholt.de', 'burgtrausnitz.de',
       'schloss-rochsburg.de', 'reichsburgcochem.de', 'landkreis-leer.de',
       'residenz-wuerzburg.de', 'schloss-weikersheim.de',
       'schloss-langenburg.de', 'moyland.de', 'schlossgreifenstein.de',
       'aetjens-park.de', 'schlossmespelbrunn.de',
       'museumwilhelms

In [138]:
# extract all rows (lines) where the value of 'domain' is ...
# then combine these into a list of pages
document = df[df['domain'] == 'museumsverbund-nordfriesland.de']['text'].tolist()

# remove new line characters, count words&characters in each line of the text
clean_document = []
word_count = 0
character_count = 0
for i in document:
    j = i.replace('\n', ' ').replace('\r', '')
    w = len(j.split())
    word_count += w
    character_count +=len(j)
    clean_document.append(j)

In [ ]:
print('word_count:', word_count, '\ncharacter_count:', character_count)
clean_document

## 2.2 Counting terms DE

In [1]:
terms_list_all = ['koloni','ungleichheit','demokratie','armut','sklave','sklaverei','diversität','inklusion','diener','arbeiter']
#term_list = ['koloni']
#term_list = ['ungleichheit','demokratie','armut']
#term_list = ['sklave','sklaverei']
#term_list = ['diversität','inklusion]
term_list = ['diener','arbeiter']

In [2]:
for term in term_list:
    df[term] = df['text'].apply(lambda x: x.lower().count(term) if isinstance(x, str) else 0)

# Filter for rows where 'kasteel' appears at least once
mask = (df[term_list] > 0).any(axis=1)

# Filter the DataFrame based on the boolean mask.
df_filtered = df[mask]

# Create the five-column table
result_df = df_filtered[['domain','url','diener','arbeiter']]
result_df.to_csv(path+'outputs/2024_filtered_output.csv', index=False)
print(result_df.to_string())

NameError: name 'df' is not defined

In [3]:
# drop columns to reset
df = df.drop(term_list, axis=1)
df.head()

NameError: name 'df' is not defined

# 3. Collocations

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/s.savicuva.nl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
# initiate bigrams and trigrams
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
# identify all collocations in the flat list of words from all documents
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_documents)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_documents)

NameError: name 'unlist_documents' is not defined

Calculate basic frequency

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

In [ ]:
# compute basic trigrams frequency
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable[:10]

Find meaningful bi- and tri-grams by filtering adjectives and nouns based on an nltk functionality

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    for word in ngram:
        if word in stopwords_ext:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

Use advanced statistical methods like the Chi-Square to identify meaninful collocations
https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# filter bigrams using chi-square
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
bigramChiTable.head()

In [ ]:
# find meaningful trigrams by filtering basic frequency table
# function to filter trigrams
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_ext:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]
filtered_tri[:10]

In [ ]:
# Chi-sqare frequency calculation for trigrams
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
trigramChiTable.head(20)

# 4. Analyse specific collocations

In [ ]:
# search for words from this list or use another list
search_words = ['diversiteit']

In [ ]:
# SCI-KIT method, produces lists of co-occurencies for specific terms
def vectorize_text(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['text'])
    return X, vectorizer

def find_collocations(text, target_words):
    words = text.split()
    collocations = []
    for i in range(len(words) - 1):
        if words[i] in target_words:
            collocations.append((words[i], words[i + 3]))
        if words[i + 1] in target_words:
            collocations.append((words[i + 3], words[i]))
    return collocations

def get_frequent_collocations(df, most_frequent_words):
    collocations = []
    for text in df['text']:
        collocations.extend(find_collocations(text, most_frequent_words))
    collocation_counts = Counter(collocations)
    frequent_collocations = {}
    for word in most_frequent_words:
        word_collocations = {collocation: count for collocation, count in collocation_counts.items() if word in collocation}
        frequent_collocations[word] = dict(islice(Counter(word_collocations).most_common(100), 100))
    return frequent_collocations

def analyze_word_collocations(df):
    X, vectorizer = vectorize_text(df)
    most_frequent_words = search_words
    frequent_collocations = get_frequent_collocations(df, most_frequent_words)
    return frequent_collocations

In [ ]:
collocations = analyze_word_collocations(df)

In [ ]:
data = []
for word, colloc_dict in collocations.items():
   for collocation, count in colloc_dict.items():
       #collocation_str = ' '.join(collocation)  # Join collocation words into a single string
       data.append([word, collocation[1], count])
collocations_df = pd.DataFrame(data, columns=['Word', 'Collocation', 'Count'])
print(collocations_df.to_markdown(index=True))

|    | Word        | Collocation           |   Count |
|---:|:------------|:----------------------|--------:|
|  0 | diversiteit | en                    |      14 |
|  1 | diversiteit | van                   |      10 |
|  2 | diversiteit | te                    |       9 |
|  3 | diversiteit | betere                |       6 |
|  4 | diversiteit | Breng                 |       6 |
|  5 | diversiteit | disciplines           |       3 |
|  6 | diversiteit | geschiedenis.         |       3 |
|  7 | diversiteit | tuinen                |       3 |
|  8 | diversiteit | Bloemschikliefhebbers |       3 |
|  9 | diversiteit | op                    |       3 |
| 10 | diversiteit | kan                   |       2 |
| 11 | diversiteit | zoals                 |       2 |
| 12 | diversiteit | Er                    |       2 |
| 13 | diversiteit | collectie             |       2 |
| 14 | diversiteit | sierteeltsector       |       2 |
| 15 | diversiteit | in                    |       2 |
| 16 | div

## Word2Vec model

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/s.savicuva.nl/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

# X is a list of tokenized texts (i.e. list of lists of tokens)
X = [word_tokenize(item) for item in df.text.tolist()]
#print(X[0:3])
model = gensim.models.Word2Vec(X, min_count=6, vector_size=200) # min_count: how many times a word appears in the corpus; size: number of dimensions

In [ ]:
model.wv.most_similar(positive=["koloniale"], topn=20)

[('fascinerende', 0.8078145980834961),
 ('omvangrijke', 0.7768810987472534),
 ('vorming', 0.7679879665374756),
 ('vroegere', 0.7625570297241211),
 ('koloniaal', 0.7485466003417969),
 ('vormgeving', 0.7475869059562683),
 ('dagelijkse', 0.7385388016700745),
 ('slavernijverleden', 0.7347381114959717),
 ('Italiaanse', 0.7336997985839844),
 ('roerige', 0.7319380044937134),
 ('mode', 0.7306028604507446),
 ('schoonheid', 0.7296142578125),
 ('interieurs', 0.7284958958625793),
 ('veranderende', 0.72608482837677),
 ('cultuurhistorie', 0.725671112537384),
 ('menselijke', 0.7247039079666138),
 ('Daendels', 0.7230746150016785),
 ('hedendaagse', 0.7212214469909668),
 ('architectuur', 0.7210280299186707),
 ('toont', 0.7208765745162964)]

## Topic Modeling